# <center> Master Project
# Running Random Forest and linear regressions

## 1 - Importing data

In [2]:
import sklearn
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils.validation import (check_array, check_consistent_length, _num_samples, column_or_1d)

#### SCE Data
Real data (tothh_Cpoly > 1)

In [7]:
SCE_realdata = pd.read_csv('C:/Users/Louis Rosset/Documents/EPFL/Master Project/Grid_limits_EV_charging/04_allocation/IOUdata/SCE/circuitfiles/SCE_ICAalldemotrees_real.csv')

In [8]:
print(SCE_realdata.shape)

(33172, 44)


#### PG&E data
Real data (tothh_Cpoly > 1)

In [9]:
PGE_realdata = pd.read_csv('C:/Users/Louis Rosset/Documents/EPFL/Master Project/Grid_limits_EV_charging/04_allocation/IOUdata/PGE/circuitfiles/PGE_ICAalldemotrees_real.csv')

In [10]:
print(PGE_realdata.shape)

(27628, 41)


## 2 - Editing data

In [11]:
SCE_realdata.columns

Index(['CircuitName', 'GEOID', 'ICL_kWphh', 'ICL_kWphh_cap', 'CircVolt_kV',
       'Length_m', 'Length_m_ctot', 'Phase_max', 'Phase_min', 'CLSmax',
       'CLSmin', 'ResCust', 'Res_pct', 'Com_pct', 'Ind_pct', 'Agr_pct',
       'Oth_pct', 'tothh_Cpoly', 'tothh_ctot', 'tothh_pct', 'tothh_perkm',
       'tothh', 'racediversity', 'black_pct', 'asian_pct', 'nlxwhite_pct',
       'latinx_pct', 'inc50kbelow_pct', 'inc150kplus_pct', 'medhhinc',
       'edavgyrs', 'ownerocc_pct', 'singleunit_pct', 'MUD_pct', 'duplex_pct',
       'smallmulti_pct', 'largemulti_pct', 'unitsavg', 'medyrbuilt',
       'hhdensity_hhsqkm', 'polexposure_pctl', 'polenvt_pctl', 'popsens_pctl',
       'lingisolation_pctl'],
      dtype='object')

In [12]:
PGE_realdata.columns

Index(['CircuitName', 'GEOID', 'ICL_kWphh', 'ICL_kWphh_cap', 'CircVolt_kV',
       'Length_m', 'Length_m_ctot', 'ICA_pct', 'ResCust', 'Res_pct', 'Com_pct',
       'Ind_pct', 'Agr_pct', 'Oth_pct', 'tothh_Cpoly', 'tothh_ctot',
       'tothh_pct', 'tothh_perkm', 'tothh', 'racediversity', 'black_pct',
       'asian_pct', 'nlxwhite_pct', 'latinx_pct', 'inc50kbelow_pct',
       'inc150kplus_pct', 'medhhinc', 'edavgyrs', 'ownerocc_pct',
       'singleunit_pct', 'MUD_pct', 'duplex_pct', 'smallmulti_pct',
       'largemulti_pct', 'unitsavg', 'medyrbuilt', 'hhdensity_hhsqkm',
       'polexposure_pctl', 'polenvt_pctl', 'popsens_pctl',
       'lingisolation_pctl'],
      dtype='object')

In [13]:
SCE_realdata_dropped = SCE_realdata.drop(columns = ['CircuitName', 'GEOID'])
PGE_realdata_dropped = PGE_realdata.drop(columns = ['CircuitName', 'GEOID'])

In [14]:
SCE_realdata_encoded = pd.get_dummies(SCE_realdata_dropped)
print(SCE_realdata_encoded.shape)

PGE_realdata_encoded = pd.get_dummies(PGE_realdata_dropped)
print(PGE_realdata_encoded.shape)

(33172, 42)
(27628, 39)


Extract y labels

In [15]:
y_SCE = SCE_realdata_encoded[['ICL_kWphh_cap']]
print(y_SCE.shape)
y_PGE = PGE_realdata_encoded[['ICL_kWphh_cap']]
print(y_PGE.shape)

(33172, 1)
(27628, 1)


Eliminate SCE data where ICL is null

In [16]:
y_SCE = y_SCE.dropna()
print(y_SCE.shape)
SCE_realdata_encoded = SCE_realdata_encoded.loc[y_SCE.index,:]
print(SCE_realdata_encoded.shape)

(31260, 1)
(31260, 42)


Create differente sets of variables/features (X)

In [17]:
# SCE Variable Sets

X_SCE_infra_cols = ['CircVolt_kV', 'Length_m', 'Length_m_ctot']

X_SCE_service_cols = ['Res_pct', 'Com_pct', 'Ind_pct', 'Agr_pct', 'Oth_pct', 'tothh_Cpoly',
                        'tothh_ctot', 'tothh_pct', 'tothh_perkm', 'ResCust']

X_SCE_demo_cols = ['tothh', 'racediversity', 'black_pct', 'asian_pct', 'nlxwhite_pct', 
                   'latinx_pct', 'inc50kbelow_pct', 'inc150kplus_pct', 'medhhinc', 'edavgyrs',
                   'polexposure_pctl', 'polenvt_pctl', 'popsens_pctl', 'lingisolation_pctl',
                   'hhdensity_hhsqkm', 'ownerocc_pct', 'MUD_pct', 'unitsavg', 'medyrbuilt']

X_SCE_house_cols = ['singleunit_pct', 'duplex_pct', 'smallmulti_pct', 'largemulti_pct'] #Housing sizes

X_SCE_all_cols = X_SCE_infra_cols + X_SCE_service_cols + X_SCE_demo_cols


# PG&E Independent Variable Sets

X_PGE_infra_cols = ['CircVolt_kV', 'Length_m', 'ICA_pct', 'Length_m_ctot']

X_PGE_service_cols = ['Res_pct', 'Com_pct', 'Ind_pct', 'Agr_pct', 'Oth_pct', 'tothh_Cpoly',
                        'tothh_ctot', 'tothh_pct', 'tothh_perkm', 'ResCust']

X_PGE_demo_cols = ['tothh', 'racediversity', 'black_pct', 'asian_pct', 'nlxwhite_pct', 
                   'latinx_pct', 'inc50kbelow_pct', 'inc150kplus_pct', 'medhhinc', 'edavgyrs',
                   'polexposure_pctl', 'polenvt_pctl', 'popsens_pctl', 'lingisolation_pctl',
                   'hhdensity_hhsqkm', 'ownerocc_pct', 'MUD_pct', 'unitsavg', 'medyrbuilt']

X_PGE_house_cols = ['singleunit_pct', 'duplex_pct', 'smallmulti_pct', 'largemulti_pct'] #Housing sizes

X_PGE_all_cols = X_PGE_infra_cols + X_PGE_service_cols + X_PGE_demo_cols

In [18]:
#Removing hhdensity_hhsqkm from set of variables
X_SCE_demo_hhdens_cols = ['tothh', 'racediversity', 'black_pct', 'asian_pct', 'nlxwhite_pct', 
                          'latinx_pct', 'inc50kbelow_pct', 'inc150kplus_pct', 'medhhinc', 'edavgyrs',
                          'polexposure_pctl', 'polenvt_pctl', 'popsens_pctl', 'lingisolation_pctl',
                          'ownerocc_pct', 'MUD_pct', 'unitsavg', 'medyrbuilt']

X_PGE_demo_hhdens_cols = ['tothh', 'racediversity', 'black_pct', 'asian_pct', 'nlxwhite_pct', 
                          'latinx_pct', 'inc50kbelow_pct', 'inc150kplus_pct', 'medhhinc', 'edavgyrs',
                          'polexposure_pctl', 'polenvt_pctl', 'popsens_pctl', 'lingisolation_pctl',
                          'ownerocc_pct', 'MUD_pct', 'unitsavg', 'medyrbuilt']

X_SCE_all_hhdens_cols = X_SCE_infra_cols + X_SCE_service_cols + X_SCE_demo_hhdens_cols

X_PGE_all_hhdens_cols = X_PGE_infra_cols + X_PGE_service_cols + X_PGE_demo_hhdens_cols

In [19]:
# Split SCE_realdata_encoded according to columns
X_all_SCE = SCE_realdata_encoded[X_SCE_all_cols].dropna()
y_all_SCE = y_SCE.loc[X_all_SCE.index,:]
X_infra_SCE = SCE_realdata_encoded[X_SCE_infra_cols].dropna()
y_infra_SCE = y_SCE.loc[X_infra_SCE.index,:]
X_service_SCE = SCE_realdata_encoded[X_SCE_service_cols].dropna()
y_service_SCE = y_SCE.loc[X_service_SCE.index,:]
X_demo_SCE = SCE_realdata_encoded[X_SCE_demo_cols].dropna()
y_demo_SCE = y_SCE.loc[X_demo_SCE.index,:]
X_house_SCE = SCE_realdata_encoded[X_SCE_house_cols].dropna()
y_house_SCE = y_SCE.loc[X_house_SCE.index,:]

# Split PGE_realdata_encoded according to columns
X_all_PGE = PGE_realdata_encoded[X_PGE_all_cols].dropna()
y_all_PGE = y_PGE.loc[X_all_PGE.index,:]
X_infra_PGE = PGE_realdata_encoded[X_PGE_infra_cols].dropna()
y_infra_PGE = y_PGE.loc[X_infra_PGE.index,:]
X_service_PGE = PGE_realdata_encoded[X_PGE_service_cols].dropna()
y_service_PGE = y_PGE.loc[X_service_PGE.index,:]
X_demo_PGE = PGE_realdata_encoded[X_PGE_demo_cols].dropna()
y_demo_PGE = y_PGE.loc[X_demo_PGE.index,:]
X_house_PGE = PGE_realdata_encoded[X_PGE_house_cols].dropna()
y_house_PGE = y_PGE.loc[X_house_PGE.index,:]

In [20]:
X_all_hhdens_SCE = SCE_realdata_encoded[X_SCE_all_hhdens_cols].dropna()
y_all_hhdens_SCE = y_SCE.loc[X_all_hhdens_SCE.index,:]

X_all_hhdens_PGE = PGE_realdata_encoded[X_PGE_all_hhdens_cols].dropna()
y_all_hhdens_PGE = y_PGE.loc[X_all_hhdens_PGE.index,:]

X_demo_hhdens_SCE = SCE_realdata_encoded[X_SCE_demo_hhdens_cols].dropna()
y_demo_hhdens_SCE = y_SCE.loc[X_demo_hhdens_SCE.index,:]

X_demo_hhdens_PGE = PGE_realdata_encoded[X_PGE_demo_hhdens_cols].dropna()
y_demo_hhdens_PGE = y_PGE.loc[X_demo_hhdens_PGE.index,:]

## 3 - Random forest regressions

In [21]:
def rf_regression(y_val, run, X, y, utility):
    """
    Run the Random Forest Regression Models
    Save CSVs of feature importances and errors
    
    Run options:
    0. all variables
    1. infra variables
    2. service variables
    3. demo variables
    4. house variables
    """
    
    cwd = os.getcwd()
    
    rf_model = RandomForestRegressor()
    
    if utility == 'SCE':
        weights = SCE_realdata_encoded.loc[X.index, 'tothh_Cpoly']

        FI_col_name = 'SCE FI with ' + run + ' Variables'
        fi_df_name = 'SCE_FeatImp_' + run + '.csv'
        fi_fig_name = 'SCE_FI_' + run + '.png'
        eval_col_name = 'SCE Eval metrics with ' + run + ' Variables'
        eval_df_name = 'SCE_RFEval_' + run + '.csv'
        
    else: # utility == 'PGE'
        weights = PGE_realdata_encoded.loc[X.index, 'tothh_Cpoly']
        
        FI_col_name = 'PG&E FI with ' + run + ' Variables'
        fi_df_name = 'PGE_FeatImp_' + run + '.csv'
        fi_fig_name = 'PGE_FI_' + run + '.png'
        eval_col_name = 'PG&E Eval metrics with ' + run + ' Variables'
        eval_df_name = 'PGE_RFEval_' + run + '.csv'

    # Fit Final Model
    rf_model.fit(X, y, sample_weight = weights)
    # Make predictions for both train and test sets
    y_pred = rf_model.predict(X)
    # Find Feature Importances
    FI = rf_model.feature_importances_
    # Evaluation metrics: weighted R^2 and weighted RMSE
    r2 = rf_model.score(X, y, sample_weight = weights)
    mse = mean_squared_error(y_pred, y, sample_weight = weights)
    rmse = np.sqrt(mse)
    
    # Save Feature Importance Data
    FI_df = pd.DataFrame(data = FI, index = X.columns, columns = [FI_col_name])
    save_to_FI = cwd + '/feat_imp/' + fi_df_name
    FI_df.to_csv(save_to_FI)
    
    # Save Feature Importance Data as Barchart
    primary_col = FI_col_name
    primary_col_name = 'normal'
    FI_df_sorted = FI_df.sort_values(by = primary_col, ascending = False)
    FI_df_sorted[primary_col].plot(kind = 'bar', figsize = (10,5))
        
    plt.xlabel('Features', fontsize = 15);
    plt.ylabel('Feature Importance', fontsize = 15)
    plt.title(FI_col_name, fontsize = 15)
    save_to_fi_barchart = cwd + '/figures/' + fi_fig_name
    plt.savefig(save_to_fi_barchart, dpi = 300, bbox_inches = 'tight');
    plt.close()
    
    # Save Evaluation Metrics Data
    eval_df = pd.DataFrame(data = [r2, mse, rmse], 
                           index = ['r2', 'mse', 'rmse'],
                                   columns = [eval_col_name])
    save_to_eval = cwd + '/eval_metrics/' + eval_df_name
    eval_df.to_csv(save_to_eval)

In [22]:
run_vals = ['all', 'infra', 'service', 'demo', 'house', "all_hhdens", "demo_hhdens"]

In [20]:
#Run random forest regression for each group of variables
#SCE
rf_regression('ICL_kWphh_cap', run_vals[0], X_all_SCE,
          y_all_SCE['ICL_kWphh_cap'], 'SCE')

rf_regression('ICL_kWphh_cap', run_vals[1], X_infra_SCE,
          y_infra_SCE['ICL_kWphh_cap'], 'SCE')

rf_regression('ICL_kWphh_cap', run_vals[2], X_service_SCE,
          y_service_SCE['ICL_kWphh_cap'], 'SCE')
    
rf_regression('ICL_kWphh_cap', run_vals[3], X_demo_SCE,
          y_demo_SCE['ICL_kWphh_cap'], 'SCE')

rf_regression('ICL_kWphh_cap', run_vals[4], X_house_SCE,
          y_house_SCE['ICL_kWphh_cap'], 'SCE')


#PGE
rf_regression('ICL_kWphh_cap', run_vals[0], X_all_PGE,
          y_all_PGE['ICL_kWphh_cap'], 'PGE')

rf_regression('ICL_kWphh_cap', run_vals[1], X_infra_PGE,
          y_infra_PGE['ICL_kWphh_cap'], 'PGE')

rf_regression('ICL_kWphh_cap', run_vals[2], X_service_PGE,
          y_service_PGE['ICL_kWphh_cap'], 'PGE')
    
rf_regression('ICL_kWphh_cap', run_vals[3], X_demo_PGE,
          y_demo_PGE['ICL_kWphh_cap'], 'PGE')

rf_regression('ICL_kWphh_cap', run_vals[4], X_house_PGE,
          y_house_PGE['ICL_kWphh_cap'], 'PGE')

In [21]:
#Running regression for all and demographic variables without household density
rf_regression('ICL_kWphh_cap', run_vals[5], X_all_hhdens_SCE,
          y_all_hhdens_SCE['ICL_kWphh_cap'], 'SCE')
rf_regression('ICL_kWphh_cap', run_vals[5], X_all_hhdens_PGE,
          y_all_hhdens_PGE['ICL_kWphh_cap'], 'PGE')

rf_regression('ICL_kWphh_cap', run_vals[6], X_demo_hhdens_SCE,
          y_demo_hhdens_SCE['ICL_kWphh_cap'], 'SCE')
rf_regression('ICL_kWphh_cap', run_vals[6], X_demo_hhdens_PGE,
          y_demo_hhdens_PGE['ICL_kWphh_cap'], 'PGE')

## 4 - Linear regressions

### Defining useful functions

In [22]:
def _check_reg_targets(y_true, y_pred, multioutput, dtype="numeric"):
    """Check that y_true and y_pred belong to the same regression task
    """
    check_consistent_length(y_true, y_pred)
    y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))

    if y_true.shape[1] != y_pred.shape[1]:
        raise ValueError("y_true and y_pred have different number of output "
                         "({0}!={1})".format(y_true.shape[1], y_pred.shape[1]))

    n_outputs = y_true.shape[1]
    allowed_multioutput_str = ('raw_values', 'uniform_average',
                               'variance_weighted')
    if isinstance(multioutput, str):
        if multioutput not in allowed_multioutput_str:
            raise ValueError("Allowed 'multioutput' string values are {}. "
                             "You provided multioutput={!r}".format(
                                 allowed_multioutput_str,
                                 multioutput))
    elif multioutput is not None:
        multioutput = check_array(multioutput, ensure_2d=False)
        if n_outputs == 1:
            raise ValueError("Custom weights are useful only in "
                             "multi-output cases.")
        elif n_outputs != len(multioutput):
            raise ValueError(("There must be equally many custom weights "
                              "(%d) as outputs (%d).") %
                             (len(multioutput), n_outputs))
    y_type = 'continuous' if n_outputs == 1 else 'continuous-multioutput'

    return y_type, y_true, y_pred, multioutput

In [23]:
def mean_squared_error_weighted_SCE(model, X_test, y_true, *,
                       sample_weight=None,
                       multioutput='uniform_average', squared=True):
    """Mean squared error regression loss
    Read more in the :ref:`User Guide <mean_squared_error>`.
    """
    sample_weight = SCE_realdata_encoded.loc[X_test.index.tolist(),'tothh_Cpoly']
    
    y_pred = model.predict(X_test)
    
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    output_errors = np.average((y_true - y_pred) ** 2, axis=0,
                               weights=sample_weight)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors if squared else np.sqrt(output_errors)
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    mse = np.average(output_errors, weights=multioutput)
    return mse if squared else np.sqrt(mse)

In [24]:
def mean_squared_error_weighted_PGE(model, X_test, y_true, *,
                       sample_weight=None,
                       multioutput='uniform_average', squared=True):
    """Mean squared error regression loss
    Read more in the :ref:`User Guide <mean_squared_error>`.
    """
    sample_weight = PGE_realdata_encoded.loc[X_test.index.tolist(),'tothh_Cpoly']
    
    y_pred = model.predict(X_test)
    
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    output_errors = np.average((y_true - y_pred) ** 2, axis=0,
                               weights=sample_weight)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors if squared else np.sqrt(output_errors)
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    mse = np.average(output_errors, weights=multioutput)
    return mse if squared else np.sqrt(mse)

In [25]:
def r2_score_weighted_SCE(model, X_test, y_true, *, sample_weight = None,
             multioutput="uniform_average"):
    """R^2 (coefficient of determination) regression score function.
    Best possible score is 1.0 and it can be negative (because the
    model can be arbitrarily worse). A constant model that always
    predicts the expected value of y, disregarding the input features,
    would get a R^2 score of 0.0.
    Read more in the :ref:`User Guide <r2_score>`.
    """
    y_pred = model.predict(X_test)
    
    sample_weight = SCE_realdata_encoded.loc[X_test.index.tolist(),'tothh_Cpoly']
    
    if _num_samples(y_pred) < 2:
        msg = "R^2 score is not well-defined with less than two samples."
        warnings.warn(msg, UndefinedMetricWarning)
        return float('nan')

    if sample_weight is not None:
        sample_weight = column_or_1d(sample_weight)
        weight = sample_weight[:, np.newaxis]
        weight = weight.flatten()
    else:
        weight = 1.
        
    numerator = np.sum(weight * (y_true - y_pred) ** 2, axis = 0)
    denominator = np.sum(weight * (y_true - np.average(
        y_true, axis=0, weights=sample_weight)) ** 2, axis = 0)
    nonzero_denominator = denominator != 0
    nonzero_numerator = numerator != 0
    valid_score = nonzero_denominator & nonzero_numerator
    output_scores = np.ones([len(y_true)])
    output_scores[valid_score] = 1 - (numerator[valid_score] /
                                      denominator[valid_score])
    # arbitrary set to zero to avoid -inf scores, having a constant
    # y_true is not interesting for scoring a regression anyway
    output_scores[nonzero_numerator & ~nonzero_denominator] = 0.
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            # return scores individually
            return output_scores
        elif multioutput == 'uniform_average':
            # passing None as weights results is uniform mean
            avg_weights = None
        elif multioutput == 'variance_weighted':
            avg_weights = denominator
            # avoid fail on constant y or one-element arrays
            if not np.any(nonzero_denominator):
                if not np.any(nonzero_numerator):
                    return 1.0
                else:
                    return 0.0
    else:
        avg_weights = multioutput

    return np.average(output_scores, weights=avg_weights)

In [26]:
def r2_score_weighted_PGE(model, X_test, y_true, *, sample_weight = None,
             multioutput="uniform_average"):
    """R^2 (coefficient of determination) regression score function.
    Best possible score is 1.0 and it can be negative (because the
    model can be arbitrarily worse). A constant model that always
    predicts the expected value of y, disregarding the input features,
    would get a R^2 score of 0.0.
    Read more in the :ref:`User Guide <r2_score>`.
    """
    y_pred = model.predict(X_test)
    
    sample_weight = PGE_realdata_encoded.loc[X_test.index.tolist(),'tothh_Cpoly']

    if _num_samples(y_pred) < 2:
        msg = "R^2 score is not well-defined with less than two samples."
        warnings.warn(msg, UndefinedMetricWarning)
        return float('nan')

    if sample_weight is not None:
        sample_weight = column_or_1d(sample_weight)
        weight = sample_weight[:, np.newaxis]
        weight = weight.flatten()
    else:
        weight = 1.
        
    numerator = np.sum(weight * (y_true - y_pred) ** 2, axis = 0)
    denominator = np.sum(weight * (y_true - np.average(
        y_true, axis=0, weights=sample_weight)) ** 2, axis = 0)
    nonzero_denominator = denominator != 0
    nonzero_numerator = numerator != 0
    valid_score = nonzero_denominator & nonzero_numerator
    output_scores = np.ones([len(y_true)])
    output_scores[valid_score] = 1 - (numerator[valid_score] /
                                      denominator[valid_score])
    # arbitrary set to zero to avoid -inf scores, having a constant
    # y_true is not interesting for scoring a regression anyway
    output_scores[nonzero_numerator & ~nonzero_denominator] = 0.
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            # return scores individually
            return output_scores
        elif multioutput == 'uniform_average':
            # passing None as weights results is uniform mean
            avg_weights = None
        elif multioutput == 'variance_weighted':
            avg_weights = denominator
            # avoid fail on constant y or one-element arrays
            if not np.any(nonzero_denominator):
                if not np.any(nonzero_numerator):
                    return 1.0
                else:
                    return 0.0
    else:
        avg_weights = multioutput

    return np.average(output_scores, weights=avg_weights)

In [27]:
def linreg(y_val, run, X, y, utility):
    """
    Run Linear Regression Methods on a Independent & Dependent Variable Set
    """
    cwd = os.getcwd()
    
    # Normalize
    X = (X - X.mean()) / X.std()
    
    if utility == 'SCE':
        sample_weights = SCE_realdata_encoded.loc[X.index.tolist(),'tothh_Cpoly']
        coef_df_name = 'SCE_LinMod_coef_' + run + '.csv'
        eval_col_name = 'SCE Eval metrics with ' + run + ' Variables'
        eval_df_name = 'SCE_LinModEval_' + run + '.csv'
    else: # utility == 'PGE'
        sample_weights = PGE_realdata_encoded.loc[X.index.tolist(),'tothh_Cpoly']
        coef_df_name = 'PGE_LinMod_coef_' + run + '.csv'
        eval_col_name = 'PG&E Eval metrics with ' + run + ' Variables'
        eval_df_name = 'PGE_LinModEval_' + run + '.csv'
    
    # Model Creation and Evaluation
    lm = LinearRegression().fit(X, y, sample_weight = sample_weights)
    if utility == 'SCE':
        mse = mean_squared_error_weighted_SCE(lm, X, y)
        rmse = np.sqrt(mse)
        r2 = r2_score_weighted_SCE(lm, X, y)
    else: # utility = 'PGE'
        mse = mean_squared_error_weighted_PGE(lm, X, y)
        rmse = np.sqrt(mse)
        r2 = r2_score_weighted_PGE(lm, X, y)

    eval_metrics = pd.DataFrame(data = [r2, 
                                        mse, 
                                        rmse],
                               index = ['r2', 
                                        'mse', 
                                        'rmse'],
                               columns = [eval_col_name])
    coef_df = pd.DataFrame(data = lm.coef_, index = X.columns, columns=['Linear Model Coefs'])
        
    y_hat = lm.predict(X)
    residuals = y - y_hat
    residenal_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residenal_sum_of_squares / (X.shape[0] - (X.shape[1]))
    var_beta_hat = np.linalg.inv(X.T @ X) * sigma_squared_hat
    standard_errors = []
    for p_ in range(X.shape[1]):
        standard_errors.append(var_beta_hat[p_, p_] ** 0.5)
        
    coef_df['Standard Errors'] = standard_errors
    
    # Save Eval Metrics and Coefficients
    save_to_eval = cwd + '/lm_eval_metrics/' + eval_df_name
    eval_metrics.to_csv(save_to_eval)
    save_to_coef = cwd + '/lm_coef/' + coef_df_name
    coef_df.to_csv(save_to_coef)

In [35]:
#Remove collinear columns
X_all_SCE_mod = X_all_SCE.drop(columns = ['Agr_pct', 'Ind_pct', 'Oth_pct'])
X_all_PGE_mod = X_all_PGE.drop(columns = ['Agr_pct', 'Ind_pct', 'Oth_pct'])
X_service_SCE_mod = X_service_SCE.drop(columns = ['Agr_pct', 'Ind_pct', 'Oth_pct'])
X_service_PGE_mod = X_service_PGE.drop(columns = ['Agr_pct', 'Ind_pct', 'Oth_pct'])

X_all_hhdens_SCE_mod = X_all_hhdens_SCE.drop(columns = ['Agr_pct', 'Ind_pct', 'Oth_pct'])
X_all_hhdens_PGE_mod = X_all_hhdens_PGE.drop(columns = ['Agr_pct', 'Ind_pct', 'Oth_pct'])

In [37]:
#Run linear regression for each group of variables
#SCE
linreg('ICL_kWphh_cap', run_vals[0], X_all_SCE_mod,
          y_all_SCE['ICL_kWphh_cap'], 'SCE')

linreg('ICL_kWphh_cap', run_vals[1], X_infra_SCE,
          y_infra_SCE['ICL_kWphh_cap'], 'SCE')

linreg('ICL_kWphh_cap', run_vals[2], X_service_SCE_mod,
          y_service_SCE['ICL_kWphh_cap'], 'SCE')
    
linreg('ICL_kWphh_cap', run_vals[3], X_demo_SCE,
          y_demo_SCE['ICL_kWphh_cap'], 'SCE')

linreg('ICL_kWphh_cap', run_vals[4], X_house_SCE,
          y_house_SCE['ICL_kWphh_cap'], 'SCE')

#PGE
linreg('ICL_kWphh_cap', run_vals[0], X_all_PGE_mod,
          y_all_PGE['ICL_kWphh_cap'], 'PGE')

linreg('ICL_kWphh_cap', run_vals[1], X_infra_PGE,
          y_infra_PGE['ICL_kWphh_cap'], 'PGE')

linreg('ICL_kWphh_cap', run_vals[2], X_service_PGE_mod,
          y_service_PGE['ICL_kWphh_cap'], 'PGE')
    
linreg('ICL_kWphh_cap', run_vals[3], X_demo_PGE,
          y_demo_PGE['ICL_kWphh_cap'], 'PGE')

linreg('ICL_kWphh_cap', run_vals[4], X_house_PGE,
          y_house_PGE['ICL_kWphh_cap'], 'PGE')

In [38]:
#Running regression for all and demographic variables without household density
linreg('ICL_kWphh_cap', run_vals[5], X_all_hhdens_SCE_mod,
          y_all_hhdens_SCE['ICL_kWphh_cap'], 'SCE')

linreg('ICL_kWphh_cap', run_vals[6], X_demo_hhdens_SCE,
          y_demo_hhdens_SCE['ICL_kWphh_cap'], 'SCE')

linreg('ICL_kWphh_cap', run_vals[5], X_all_hhdens_PGE_mod,
          y_all_hhdens_PGE['ICL_kWphh_cap'], 'PGE')

linreg('ICL_kWphh_cap', run_vals[6], X_demo_hhdens_PGE,
          y_demo_hhdens_PGE['ICL_kWphh_cap'], 'PGE')